In [1]:
import pyvo
import numpy
from tabulate import tabulate
import pandas
import warnings
import matplotlib
warnings.filterwarnings("ignore", category=UserWarning)
import math
service = pyvo.dal.TAPService("http://voparis-tap-planeto.obspm.fr/tap")
import seaborn

import matplotlib.pyplot as plt
from astropy import units
from astropy import constants
from astropy.visualization import quantity_support
from matplotlib_inline.backend_inline import set_matplotlib_formats
from matplotlib import cm
import matplotlib.colors
import matplotlib.ticker
from scipy import stats # For in-built method to get PCC
import scipy.optimize
from scipy import interpolate
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.integrate import trapz
from scipy.stats import norm


from matplotlib.colors import ListedColormap
from matplotlib.colors import LogNorm

from collections import OrderedDict
from matplotlib.lines import Line2D
import itertools
from collections import OrderedDict
from functools import partial
import matplotlib.ticker as mticker
from cycler import cycler
mass_coeff = constants.M_jup / constants.M_earth
radius_coeff = constants.R_jup / constants.R_earth
pandas.options.mode.use_inf_as_na = True

quantity_support()
set_matplotlib_formats('svg')


curves = pandas.read_fwf("../data/curves_all.txt", index_col=0)#, header=None)

pandas.options.mode.use_inf_as_na = True

def lookForParameterInNASA(planetName, param):
    queryNASA = (
        " ".join((
            "SELECT * FROM",
            "(WITH latestEntries AS",
            f"(SELECT {', '.join(fieldsNASA)}, {param},",
            f"ROW_NUMBER() OVER(PARTITION BY pl_name ORDER BY CASE WHEN {param} IS NULL THEN 1 ELSE 0 END, pl_pubdate DESC) AS rank",
            f"FROM {tableNameNASA}",
            f"WHERE LOWER(pl_name) = LOWER('{planetName}'))",
            f"SELECT {param} FROM latestEntries WHERE rank = 1 AND {param} IS NOT NULL)"
        ))
    )
    resultsNASA = serviceNASA.search(queryNASA)
    if len(resultsNASA) != 0:
        return resultsNASA[0].get(param)
    else:
        return None